# 环境配置

In [24]:
!nvidia-smi

Tue Apr  1 08:25:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
!uname -m && cat /etc/*release

x86_64
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"
PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [26]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [27]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers==0.0.25
!pip install .[bitsandbytes]
!pip install -e ".[torch,metrics,bitsandbytes]"

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (285/285), done.
remote: Total 346 (delta 80), reused 168 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (346/346), 9.54 MiB | 19.41 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-5p_mv1_4/unsloth_e58b50382dc142c2bde31bd53694639b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-5p_mv1_4/unsloth_e58b50382dc142c2bde31bd53694639b
  Resolved https://github.com/unslothai/unsloth.git to commit b44f89e60ac7ec54fdc18a58e8f0905c4d6a79f2
  Inst

如果出现依赖错误，请运行一下命令行

In [5]:
!pip install --no-deps -e .

Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.3.dev0-0.editable-py3-none-any.whl size=26073 sha256=08952c8f5170b37b3a67d8dc8280218e0c8bfa16cf69db9688a3c5dfa3ac48c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-4_zaaom7/wheels/bd/34/05/1e3cb4b8f20c20631b411dc5157b4b150850c03496fa96c2c4
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.9.3.dev0
    Uninstalling llamafactory-0.9.3.dev0:
      Successfully uninstalled llamafactory-0.9.3.dev0


In [28]:
!llamafactory-cli version

2025-04-01 08:26:53.766254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743496013.789070   18442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743496013.795363   18442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 08:26:53.820415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.3.dev0           |
|      

# 修改认知数据集

In [29]:
import json

%cd /content/LLaMA-Factory/

NAME = "超级无敌鸿蒙机器人"
AUTHOR = "Sun Guyi"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


# **添加微调的训练集，并将其注册到数据集中**



# 进行微调

In [31]:
import json

args = dict(
  stage="sft",                                               # 进行指令监督微调
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  dataset="identity,data0",          # 使用 alpaca 和自我认知数据集
  template="llama3",                                         # 使用 llama3 提示词模板
  finetuning_type="lora",                                    # 使用 LoRA 适配器来节省显存
  lora_target="all",                                         # 添加 LoRA 适配器至全部线性层
  output_dir="llama3_lora",                                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,                             # 批处理大小
  gradient_accumulation_steps=4,                             # 梯度累积步数
  lr_scheduler_type="cosine",                                # 使用余弦学习率退火算法
  logging_steps=5,                                           # 每 5 步输出一个记录
  warmup_ratio=0.1,                                          # 使用预热学习率
  save_steps=1000,                                           # 每 1000 步保存一个检查点
  learning_rate=5e-5,                                        # 学习率大小
  num_train_epochs=3.0,                                      # 训练轮数
  max_samples=300,                                           # 使用每个数据集中的 300 条样本
  max_grad_norm=1.0,                                         # 将梯度范数裁剪至 1.0
  loraplus_lr_ratio=16.0,                                    # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                                                 # 使用 float16 混合精度训练
  report_to="none",                                          # 关闭 wandb 记录器
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

/content/LLaMA-Factory
2025-04-01 08:28:59.294566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743496139.316459   18984 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743496139.322927   18984 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 08:28:59.345136: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[INFO|2025-04-01 08:29:07] llamafactory.hparams.parser:383 >> Process rank: 0, world size: 1, device: 

# 使用webUI进行对话

In [32]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2025-04-01 09:02:08.534064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743498128.556253   27347 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743498128.562930   27347 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 09:02:08.588696: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on public URL: https://c00fb5309b96324759.gradio.live
[WARNING|2025-04-01 09:06:00] llamafac